<a href="https://colab.research.google.com/github/psse-cpu/ml-workshop/blob/main/notebooks/playground/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=3, suppress=True, linewidth=320)

tf.__version__

'2.8.0'

In [35]:
mnist = keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

In [36]:
training_images.shape, training_labels.shape

((60000, 28, 28), (60000,))

In [37]:
test_images.shape, test_labels.shape

((10000, 28, 28), (10000,))

In [38]:
training_images_norm = training_images / 255.0
test_images_norm = test_images / 255.0

In [39]:
training_images_norm[0]

array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.004, 0.   , 0.   , 0.051, 0.286, 0.   , 0.   , 0.004, 0.016, 0.   , 0.   , 0.   , 0.   , 0.004, 0.004, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.012, 0.   , 0.141, 0.533, 0.498, 0.243, 0.212, 0.   , 0.   , 0.   , 0.004, 0.012, 0.01

In [40]:
test_images_norm[0]

array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.  

In [41]:
def show_info(model):
    model.summary()

    for layer in model.layers:
        if (isinstance(layer, keras.layers.Dense)):
            print(f'Layer name: {layer.name};\tActivation: {layer.activation.__name__}')
        else:
            print(f'Layer name: {layer.name}')

    print(f'Optimizer: {model.optimizer}')

def train(**kwargs):
    # default activation for hidden layers is sigmoid
    hidden_layers_activation = kwargs.get('hidden_layers_activation', 'sigmoid')
    optimizer = kwargs.get('optimizer', keras.optimizers.SGD())
    normalize = kwargs.get('normalize', True)
    epochs = kwargs.get('epochs', 5)

    # default to one hidden layer with 128 neurons
    hidden_layers_units = kwargs.get('hidden_layers_units', [128])
    hidden_layers = [
        keras.layers.Dense(units, hidden_layers_activation) 
        for units in hidden_layers_units
    ]

    X_train = training_images_norm if normalize else training_images
    X_test = test_images_norm if normalize else test_images

    model = keras.models.Sequential([
        keras.layers.Flatten(),
        *hidden_layers, 
        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer, loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
    model.fit(X_train, training_labels, epochs=epochs)
    show_info(model)

    model.evaluate(X_test, test_labels)

In [42]:
%%time
train(hidden_layers_units=[128])

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.2652 - accuracy: 0.6677
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.7650 - accuracy: 0.7524
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.6586 - accuracy: 0.7749
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.6033 - accuracy: 0.7918
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5665 - accuracy: 0.8044
Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_17 (Flatten)        (32, 784)                 0         
                                                                 
 dense_37 (Dense)            (32, 128)                 100480    
                                                                 
 dense_38 (Dense)            (32, 10)                  1290      
          

In [43]:
%%time

train(hidden_layers_activation='relu')

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.7407 - accuracy: 0.7636
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5101 - accuracy: 0.8260
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4655 - accuracy: 0.8409
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4416 - accuracy: 0.8474
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4242 - accuracy: 0.8534
Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_18 (Flatten)        (32, 784)                 0         
                                                                 
 dense_39 (Dense)            (32, 128)                 100480    
                                                                 
 dense_40 (Dense)            (32, 10)                  1290      
          

In [44]:
%%time

train(optimizer=keras.optimizers.Adam())

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5451 - accuracy: 0.8134
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3922 - accuracy: 0.8582
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3554 - accuracy: 0.8720
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3300 - accuracy: 0.8799
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3100 - accuracy: 0.8876
Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_19 (Flatten)        (32, 784)                 0         
                                                                 
 dense_41 (Dense)            (32, 128)                 100480    
                                                                 
 dense_42 (Dense)            (32, 10)                  1290      
          

In [45]:
%%time

train(hidden_layers_activation='relu', optimizer=keras.optimizers.Adam())

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5015 - accuracy: 0.8242
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3776 - accuracy: 0.8648
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3390 - accuracy: 0.8774
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3140 - accuracy: 0.8857
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2973 - accuracy: 0.8907
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_20 (Flatten)        (32, 784)                 0         
                                                                 
 dense_43 (Dense)            (32, 128)                 100480    
                                                                 
 dense_44 (Dense)            (32, 10)                  1290      
          

In [46]:
%%time

train(
    hidden_layers_activation='relu', 
    optimizer=keras.optimizers.Adam(),
    hidden_layers_units=[512]
)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4753 - accuracy: 0.8303
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3542 - accuracy: 0.8699
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3212 - accuracy: 0.8812
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2963 - accuracy: 0.8893
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2786 - accuracy: 0.8965
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_21 (Flatten)        (32, 784)                 0         
                                                                 
 dense_45 (Dense)            (32, 512)                 401920    
                                                                 
 dense_46 (Dense)            (32, 10)                  5130      
          

In [47]:
%%time

train(
    hidden_layers_activation='relu', 
    optimizer=keras.optimizers.Adam(),
    hidden_layers_units=[1024]
)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4720 - accuracy: 0.8308
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3596 - accuracy: 0.8685
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3194 - accuracy: 0.8820
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2970 - accuracy: 0.8882
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2791 - accuracy: 0.8951
Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_22 (Flatten)        (32, 784)                 0         
                                                                 
 dense_47 (Dense)            (32, 1024)                803840    
                                                                 
 dense_48 (Dense)            (32, 10)                  10250     
          

In [48]:
%%time

train(
    hidden_layers_activation='relu', 
    optimizer=keras.optimizers.Adam(),
    hidden_layers_units=[128],
    epochs=15
)

Epoch 1/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4977 - accuracy: 0.8253
Epoch 2/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3767 - accuracy: 0.8642
Epoch 3/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3392 - accuracy: 0.8762
Epoch 4/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3172 - accuracy: 0.8844
Epoch 5/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2981 - accuracy: 0.8894
Epoch 6/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2813 - accuracy: 0.8964
Epoch 7/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2711 - accuracy: 0.8996
Epoch 8/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2573 - accuracy: 0.9036
Epoch 9/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2505 - accuracy: 0.9067
Epoch 10/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.239

In [33]:
%%time

train(
    hidden_layers_activation='relu', 
    optimizer=keras.optimizers.Adam(),
    hidden_layers_units=[128, 128],
    epochs=30
)

Epoch 1/30
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4876 - accuracy: 0.8247
Epoch 2/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3626 - accuracy: 0.8674
Epoch 3/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3271 - accuracy: 0.8780
Epoch 4/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3058 - accuracy: 0.8869
Epoch 5/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2873 - accuracy: 0.8927
Epoch 6/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2715 - accuracy: 0.8974
Epoch 7/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2632 - accuracy: 0.9014
Epoch 8/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2494 - accuracy: 0.9051
Epoch 9/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2425 - accuracy: 0.9085
Epoch 10/30
1875/1875 [==============================] - 5s 3ms/step - loss: 0.232